In [1]:
!pip install pymupdf pandas openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 40.4 MB/s eta 0:00:00


In [2]:
import os, re, fitz, pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter

# ---------------- Helpers umum ----------------
def parse_amount_eu(s: str) -> float:
    """Konversi '4.411,79' -> 4411.79 (format EU)."""
    if not s:
        return 0.0
    try:
        return float(s.replace('.', '').replace(',', '.'))
    except ValueError:
        return 0.0

def parse_float_dot_or_comma(s: str) -> float:
    """Untuk KGS/CBM dengan koma/titik desimal."""
    return float(s.replace(',', '.'))

def find_one(pat: re.Pattern, text: str, default="N/A") -> str:
    m = pat.search(text)
    return m.group(1).strip() if m else default

def sum_matches(pat: re.Pattern, text: str, ndigits=3) -> float:
    vals = [parse_float_dot_or_comma(v) for v in pat.findall(text)]
    return round(sum(vals), ndigits) if vals else 0.0

def canonicalize_label(s: str) -> str:
    s = s.replace("\n", " ")
    s = re.sub(r"\(.*?\)", "", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def map_label(lbl: str) -> str:
    L = lbl.lower()
    # --- mapping khusus (sesuai permintaan) ---
    if "grundbetrag umrechnungskurs" in L: return "ENS"
    if "unsere leistungen grundbetrag" in L: return "DROP"
    # --- mapping umum ---
    if "seefracht" in L: return "SFRT"
    if "thc" in L: return "THC"
    if "abfertigungskosten" in L or "empfangshafen" in L: return "CCDE"
    if "isps" in L: return "ISPS"
    if "summarische eingangsmeldung" in L or re.search(r"\bens\b", L): return "ENS"
    if L.startswith("zoll lt. auslage"): return "ZOAB"
    if L.startswith("eust lt. auslage"): return "EUST"
    if "delivery-/drop-off-gebühr" in L: return "DROP"
    if "nachlaufkosten" in L: return "NL"
    if "importverzollung" in L: return "Zoll"
    return lbl  # fallback

# ---------------- Regex bank ----------------
R = {
    "invoice_no": re.compile(r"Rechnung(?:s|-)? ?(?:Nr\.)?:?\s*([0-9]+)", re.I),
    # (Masih dipakai untuk fallback terbatas, namun utama pakai find_invoice_date)
    "invoice_date_hint": re.compile(r"Rechnungsdatum:\s*([0-9]{2}-[A-Za-zÄÖÜäöüß]{3}-[0-9]{4})", re.I),
    "stt": re.compile(r"STT[-\s]?Nr\.?:\s*([0-9]+)", re.I),
    "sender": re.compile(r"Absender:\s*([^\n\r]+)", re.I),
    "etd_eta": re.compile(r"ETD\s*/\s*ETA:\s*([^\n\r]+)", re.I),
    "pol": re.compile(r"Port of Loading:\s*([^\n\r]+)", re.I),
    "pod": re.compile(r"Port of Discharge:\s*([^\n\r]+)", re.I),

    # KGS/CBM tiga desimal (umum di dokumen)
    "kgs": re.compile(r"([0-9]{1,6}[.,][0-9]{3})\s*KGS", re.I),
    "cbm": re.compile(r"([0-9]{1,3}[.,][0-9]{3})\s*CBM", re.I),

    # Label 1–2 baris, lalu 0–4 baris info (USD/kurs/etc), lalu baris 'EUR a EUR b'
    "cost_block": re.compile(
        r"([^\n]{3,60}(?:\n[^\n]{2,60})?)"        # label 1–2 baris
        r"(?:\n.*?){0,4}?"                        # hingga 4 baris pengantar
        r"\n\s*EUR\s*([0-9.,]+)\s*EUR\s*([0-9.,]+)",  # dua angka EUR
        re.I
    ),
}

# ----------- Regex tanggal fleksibel + finder -----------
DATE_ANY = (
    r"(?:"
    r"\d{1,2}[./-]\d{1,2}[./-]\d{4}"  # 27.09.2025 / 27-09-2025
    r"|"
    r"\d{1,2}[./\-\s]?(?:Jan|Feb|Mär|Mrz|Mar|Apr|Mai|May|Jun|Jul|Aug|Sep|Sept|Okt|Oct|Nov|Dez|Dec)[./\-\s]\d{4}"  # 27 Sep 2025 / 27-Sep-2025
    r"|"
    r"\d{4}[./-]\d{1,2}[./-]\d{1,2}"  # 2025-09-27
    r")"
)
DATE_ANY_RE = re.compile(DATE_ANY, re.I)
INVOICE_DATE_LINE = re.compile(
    r"(?:Rechnungsdatum|Invoice\s*Date)\s*[:\-]?\s*(?:\n|\r|\s){0,20}(" + DATE_ANY + r")",
    re.I
)

def find_invoice_date(text: str) -> str:
    # 1) Label + tanggal (boleh beda baris/ada spasi)
    m = INVOICE_DATE_LINE.search(text)
    if m:
        return m.group(1).strip()

    # 2) Fallback: cari label lalu scan 1–6 baris berikutnya
    lines = text.splitlines()
    for i, line in enumerate(lines):
        if re.search(r"Rechnungsdatum|Invoice\s*Date", line, re.I):
            for j in range(1, 7):
                if i + j < len(lines):
                    n = DATE_ANY_RE.search(lines[i + j])
                    if n:
                        return n.group(0).strip()

    # 3) Fallback lama (jika kebetulan format persis)
    m2 = R["invoice_date_hint"].search(text)
    if m2:
        return m2.group(1).strip()

    return "N/A"

# ---------------- Core extraction ----------------
def extract_rows_from_pdf(path: str):
    with fitz.open(path) as doc:
        text = "".join(page.get_text() for page in doc)

    meta = {
        "file": os.path.basename(path),
        "invoice_number": find_one(R["invoice_no"], text, "N/A"),
        "invoice_date": find_invoice_date(text),
        "stt_number": find_one(R["stt"], text, "N/A"),
        "sender": find_one(R["sender"], text, "N/A"),
        "etd_eta": find_one(R["etd_eta"], text, "N/A"),
        "port_loading": find_one(R["pol"], text, "N/A"),
        "port_discharge": find_one(R["pod"], text, "N/A"),
        "gross_weight_kg": sum_matches(R["kgs"], text),  # numeric float
        "volume_cbm": sum_matches(R["cbm"], text),       # numeric float
    }

    rows = []
    for raw_label, _base, amount in R["cost_block"].findall(text):
        clean = canonicalize_label(raw_label)
        mapped = map_label(clean)
        rows.append({
            **meta,
            "cost_type": mapped,
            "amount": parse_amount_eu(amount),  # numeric float
        })
    return rows

# ---------------- Excel styling ----------------
def style_excel(path: str):
    wb = load_workbook(path)
    ws = wb.active

    # Header style
    header_font = Font(bold=True, color="FFFFFF")
    header_fill = PatternFill("solid", fgColor="4F81BD")
    for c in ws[1]:
        c.font = header_font
        c.fill = header_fill
        c.alignment = Alignment(horizontal="center", vertical="center")

    # Auto width (maks 60)
    for col_cells in ws.columns:
        length = max(len(str(c.value)) if c.value is not None else 0 for c in col_cells)
        ws.column_dimensions[get_column_letter(col_cells[0].column)].width = min(length + 2, 60)

    # Rata kanan untuk angka
    for row in ws.iter_rows(min_row=2):
        for cell in row:
            if isinstance(cell.value, (int, float)):
                cell.alignment = Alignment(horizontal="right")

    ws.freeze_panes = "A2"
    ws.auto_filter.ref = ws.dimensions
    wb.save(path)

# ---------------- Upload (Colab) ----------------
try:
    from google.colab import files as gfiles
    print("Silakan upload 1 atau beberapa PDF invoice…")
    uploaded = gfiles.upload()  # dialog upload
    pdf_paths = [n for n in uploaded.keys() if n.lower().endswith(".pdf")]
    if not pdf_paths:
        raise ValueError("Tidak ada file PDF yang diupload.")
except Exception as e:
    # Fallback: jika tidak di Colab atau upload gagal, gunakan path lokal
    print("Upload dilewati / gagal:", e)

# ---------------- Run extraction ----------------
all_rows = []
for p in pdf_paths:
    try:
        all_rows.extend(extract_rows_from_pdf(p))
    except Exception as e:
        print(f"Gagal proses {p}: {e}")

df = pd.DataFrame(all_rows)

# Pastikan urutan kolom rapi:
cols_order = [
    "file", "invoice_number", "invoice_date", "stt_number",
    "sender", "etd_eta", "port_loading", "port_discharge",
    "gross_weight_kg", "volume_cbm", "cost_type", "amount"
]
df = df[[c for c in cols_order if c in df.columns]]

out_path = "invoice_long_format.xlsx"
df.to_excel(out_path, index=False)
style_excel(out_path)

# ---------------- Download (Colab) ----------------
try:
    gfiles.download(out_path)
except Exception as e:
    print("Auto-download gagal:", e)
    print("File tersimpan di:", os.path.abspath(out_path))

print(f"Selesai. Total baris: {len(df)} | File: {out_path}")


Silakan upload 1 atau beberapa PDF invoice…


Saving 3506543810.pdf to 3506543810.pdf
Saving 3506605312.pdf to 3506605312.pdf
Saving 3506605334.pdf to 3506605334.pdf
Saving 3506605355.pdf to 3506605355.pdf
Saving 3506605375.pdf to 3506605375.pdf
Saving 3506605442.pdf to 3506605442.pdf
Saving 3506605459.pdf to 3506605459.pdf
Saving 3506605469.pdf to 3506605469.pdf
Saving 3506605553.pdf to 3506605553.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Selesai. Total baris: 91 | File: invoice_long_format.xlsx
